In [147]:
import psycopg2
import sys
import os
import pandas as pd
import math

In [89]:
HOST="htm-redhorse.cpgyjyre5uq9.us-west-2.rds.amazonaws.com"
DATABASE="htm_redhorse_postgres"
USER="westinn"
PASSWORD="rootroot"

#Define our connection string
conn_string = "host='"+ HOST + "' dbname='" + DATABASE +"' user='" + USER + "' password='" + PASSWORD +"'"

# print the connection string we will use to connect
print("Connecting to database\n    ->%s" % (conn_string))

# # get a connection, if a connect cannot be made an exception will be raised here
# conn = psycopg2.connect(conn_string)

# # conn.cursor will return a cursor object, you can use this cursor to perform queries
# cursor = conn.cursor()
# print("Connected!\n")

# connection should be closed after the query
# if sql fails

def execute(query):
   try:
       conn = psycopg2.connect(conn_string)
       cursor = conn.cursor()
       print("Query:")
       print(query)
       df = pd.io.sql.read_sql_query(query, conn) # get all rows with mmsi
       conn.close()
       return df
   except Exception as err:
       print("Error:")
       print(err)
       cursor.execute("rollback")
       conn.close()

Connecting to database
    ->host='htm-redhorse.cpgyjyre5uq9.us-west-2.rds.amazonaws.com' dbname='htm_redhorse_postgres' user='westinn' password='rootroot'


In [96]:
query = """
    select *
     from analysis as t1, analysis as t2
     where ST_Intersects(t1."GEOM", t2."GEOM") = true
        and t1."min_hour_BaseDate" = t2."min_hour_BaseDate"
        and t1."MMSI" < t2."MMSI"
        and t1."min_SOG" > 1.0 and t2."min_SOG" > 1.0
    limit 1
;
"""

df = execute(query)

Query:

    select *
     from analysis as t1, analysis as t2
     where ST_Intersects(t1."GEOM", t2."GEOM") = true
        and t1."min_hour_BaseDate" = t2."min_hour_BaseDate"
        and t1."MMSI" < t2."MMSI"
        and t1."min_SOG" > 1.0 and t2."min_SOG" > 1.0
    limit 1
;



In [97]:
df

,MMSI,min_LAT,max_LAT,min_LON,max_LON,min_BaseDate,max_BaseDate,min_hour_BaseDate,min_SOG,max_SOG,...,max_LON,min_BaseDate,max_BaseDate,min_hour_BaseDate,min_SOG,max_SOG,GEOM,points,COGS,SOGS
0,538003095,25.18014,25.40751,-79.64881,-79.6033,2017-01-15 01:00:30,2017-01-15 01:58:55,2017-01-15 01:00:00,14.0,14.4,...,-79.66935,2017-01-15 01:00:16,2017-01-15 01:59:17,2017-01-15 01:00:00,12.2,12.4,0103000020E610000001000000AA000000747E1FED1CE8...,"[[-79.67266, 25.29821], [-79.67274, 25.29417],...","[181.3, 180.9, 181.2, 180.7, 180.7, 180.8, 180...","[12.2, 12.2, 12.2, 12.3, 12.4, 12.3, 12.3, 12...."


In [121]:
df_v1 = df.iloc[0,0:14]
df_v2 = df.iloc[0,14:]

df_v2

MMSI                                                         636015295
min_LAT                                                        25.0965
max_LAT                                                        25.2982
min_LON                                                       -79.6737
max_LON                                                       -79.6693
min_BaseDate                                       2017-01-15 01:00:16
max_BaseDate                                       2017-01-15 01:59:17
min_hour_BaseDate                                  2017-01-15 01:00:00
min_SOG                                                           12.2
max_SOG                                                           12.4
GEOM                 0103000020E610000001000000AA000000747E1FED1CE8...
points               [[-79.67266, 25.29821], [-79.67274, 25.29417],...
COGS                 [181.3, 180.9, 181.2, 180.7, 180.7, 180.8, 180...
SOGS                 [12.2, 12.2, 12.2, 12.3, 12.4, 12.3, 12.3, 12....
Name: 

## Check if a vessel is a giveway or standon

In [141]:
# Check if a vessel is a giveway or standon

import statistics
import numpy as np

def SG_function(SOG_Vessel,COG_Vessel):
    speed_diff = []
    cog_diff = []
    for i in range(len(SOG_Vessel)-1):
        speed_diff.append(abs(SOG_Vessel[i]-SOG_Vessel[i+1]))
        cog_diff.append(abs(COG_Vessel[i]-COG_Vessel[i+1]))
    if max(speed_diff) <= 3 and max(cog_diff) <= 30:
        sg = 'stand on'
    else:
        sg = 'giveway'
    return sg;

In [142]:
SOG_Vessel = [15, 15.3, 15.1, 15.9] 
COG_Vessel = [100,104,101,112] 

SG_function(SOG_Vessel,COG_Vessel)

'stand on'

## Calculate the great circle distance between two points on the earth (specified in decimal degrees)


In [35]:
# Calculate the great circle distance between two points on the earth (specified in decimal degrees)
lon1 = -117.22913
lat1 = 32.70419
lon2 = -117.22731
lat2 = 32.70577

#32.70419	-117.22913
#32.70577	-117.22731



def haversine(lat1, lon1, lat2, lon2):

      R = 3959.87433 # this is in miles.  For Earth radius in kilometers use 6372.8 km

      dLat = radians(lat2 - lat1)
      dLon = radians(lon2 - lon1)
      lat1 = radians(lat1)
      lat2 = radians(lat2)

      a = sin(dLat/2)**2 + cos(lat1)*cos(lat2)*sin(dLon/2)**2
      c = 2*asin(sqrt(a))

      return R * c

print(haversine(lat1, lon1, lat2, lon2))

0.1520762544582352


## Calculate SOG

In [6]:
from datetime import datetime

def dAtD(dateT):
    dt = dateT.split('T')
    dta = dt[0].split('-') + dt[1].split(':')
    for i in dta:
        dta[dta.index(i)] = int(i)
    dta = datetime(dta[0],dta[1],dta[2],dta[3],dta[4],dta[5])
    return dta


def sogCalc(haversineOutput, time1, time2):
    """Input haversine output [in miles] and the raw data for time1 and time2"""
    haversineOutput *= 0.868976
    timeDif = dAtD(time2)-dAtD(time1)
    return haversineOutput / (timeDif.total_seconds()/3600)
    
s = sogCalc( 10, "2017-01-03T22:09:25", "2017-01-03T23:00:26")
print(s)

10.219907219862788


## Calculate Bearing

In [26]:
# Calculate Bearing
from math import *

lon1 = -117.22913
lat1 = 32.70419
lon2 = -117.22731
lat2 = 32.70577

#32.70419	-117.22913
#32.70577	-117.22731
def calc_bearing(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    Base = 6371 * c


    Bearing =atan2(cos(lat1)*sin(lat2)-sin(lat1)*cos(lat2)*cos(lon2-lon1), sin(lon2-lon1)*cos(lat2)) 

    Bearing = degrees(Bearing)
    return Bearing
    #print(Bearing)

calc_bearing(lon1, lat1, lon2, lat2)

45.89417815796708

## Label Head-on colregs

In [128]:
def label_headon(cog_vessel_1,cog_vessel_2,sog_vessel_1, sog_vessel_2,lat_vessel_1,lon_vessel_1,lat_vessel_2,lon_vessel_2):
    cog_vessel_1_up = cog_vessel_1 + 7 
    cog_vessel_1_down = cog_vessel_1 - 7 
    cog_vessel_2_up = cog_vessel_2 + 7 
    cog_vessel_2_down = cog_vessel_2 - 7 
    
    if cog_vessel_1 <= cog_vessel_2_up and cog_vessel_1 >= cog_vessel_2_down:
        print('Same Direction! Overtaking???')
        colreg_type = 'NA'
        return colreg_type
    bearing_V1_to_V2 = calc_bearing(lon_vessel_1, lat_vessel_1, lon_vessel_2, lat_vessel_2)
    bearing_V2_to_V1 = calc_bearing(lon_vessel_2, lat_vessel_2, lon_vessel_1, lat_vessel_1)
    if bearing_V1_to_V2 <= cog_vessel_1_up and bearing_V1_to_V2 >= cog_vessel_1_down:
        if bearing_V2_to_V1 <= cog_vessel_2_up and bearing_V2_to_V1 >= cog_vessel_2_down:
            print('Head on')
            colreg_type = 'head on'
            return colreg_type
    colreg_type = 'NA'
    return colreg_type

In [129]:
cog_vessel_1 =  df_v1['COGS'][0]
cog_vessel_2 = df_v2['COGS'][0]
sog_vessel_1 = df_v1['SOGS'][0]
sog_vessel_2 = df_v2['SOGS'][0]
lat_vessel_1 = df_v1['points'][0][1]
lat_vessel_2 = df_v2['points'][0][1]
lon_vessel_1 = df_v1['points'][0][0]
lon_vessel_2 = df_v2['points'][0][0]


label_headon(cog_vessel_1,cog_vessel_2,sog_vessel_1, sog_vessel_2,lat_vessel_1,lon_vessel_1,lat_vessel_2,lon_vessel_2)

'NA'

## Label Overtaking colregs

In [202]:
def label_overtaking(lat_vessel_1_ARRAY, lon_vessel_1_ARRAY,lat_vessel_2_ARRAY,lon_vessel_2_ARRAY,cog_vessel_1_ARRAY,cog_vessel_2_ARRAY,sog_vessel_1_ARRAY, sog_vessel_2_ARRAY):
    # Find out if the vessels are stand on or giveway
    vessel_type_1 = SG_function(sog_vessel_1_ARRAY,cog_vessel_1_ARRAY)
    vessel_type_2 = SG_function(sog_vessel_2_ARRAY,cog_vessel_2_ARRAY)
    # Determine which vessel is the front vessel
    if vessel_type_1 == 'giveaway' and vessel_type_2 == 'giveaway':
        print('not overtaking')
        return
    if vessel_type_1 == 'stand on':
        front_vessel_cog = cog_vessel_1_ARRAY
        front_vessel_sog = cog_vessel_1_ARRAY
        front_vessel_lat = lat_vessel_1_ARRAY
        front_vessel_lon = lon_vessel_1_ARRAY
        back_vessel_cog = cog_vessel_2_ARRAY
        back_vessel_sog = cog_vessel_2_ARRAY
        back_vessel_lat = lat_vessel_2_ARRAY
        back_vessel_lon = lon_vessel_2_ARRAY
        print('Vessel 1 is the front vessel')
    else:
        front_vessel_cog = cog_vessel_2_ARRAY
        front_vessel_sog = cog_vessel_2_ARRAY
        front_vessel_lat = lat_vessel_2_ARRAY
        front_vessel_lon = lon_vessel_2_ARRAY
        back_vessel_cog = cog_vessel_1_ARRAY
        back_vessel_sog = cog_vessel_1_ARRAY
        back_vessel_lat = lat_vessel_1_ARRAY
        back_vessel_lon = lon_vessel_1_ARRAY
        print('Vessel 2 is the front vessel')
    
    def is_shipb_front_or_back(shipA_coor, shipB_coor):
        # create back triangle at beginning of interaction
        lat_diff = shipB_coor[1] - shipA_coor[1]
        lon_diff = shipB_coor[0] - shipA_coor[0]

        back_position_vector = [float(lon_diff), float(lat_diff)]

        magnitude = lambda vec : math.sqrt(math.pow(vec[0], 2) + math.pow(vec[1], 2))
        back_vector_mag = magnitude(back_position_vector)
        back_unit_vector = [back_position_vector[0]/back_vector_mag, back_position_vector[1]/back_vector_mag]
        back_angle = math.degrees(math.atan2(back_unit_vector[1], back_unit_vector[0]))

        print(front_vessel_cog)

        front_vessel_cog_unitcircled = 0
        # left side
        if -180 >= front_vessel_cog >= 0:
            front_vessel_cog_unitcircled = (-1 * front_vessel_cog) + 90;
        # top right quadrent
        elif 90 >= front_vessel_cog >= 0:
            front_vessel_cog_unitcircled = 90 - front_vessel_cog;
        # bottom right quadrent
        elif 180 >= front_vessel_cog >= 90:
            front_vessel_cog_unitcircled = 180 + front_vessel_cog;

        if 247.5 >= math.abs(back_angle - front_vessel_cog_unitcircled) >= 112.5:
            # ship B is behind
            print("ship B is behind")
            return false
        elif (135/2) >= math.abs(back_angle - front_vessel_cog_unitcircled):
            # ship B is in front
            print("ship B is in front")
            return true
        else:
            return None

    # check if both cogs are within 15 degrees of each other at beginning of interactions
    cog_diff_begin = front_vessel_cog[0] - back_vessel_cog[0]
    if cog_diff_begin > 30:
        print('not overtaking')
        return
    
    # check if back vessel is in back triangle at beginning of interaction
    shipA_coor = [front_vessel_lon[0], front_vessel_lat[0]]
    shipB_coor = [back_vessel_lon[0], back_vessel_lat[0]]
    result = is_shipb_front_or_back(shipA_coor, shipB_coor) 
    # True: shipB is in front of shipA
    # False: shipB is behind shipA
    # Null: Not over taking
    if result not False:
        colreg_type = 'NA'
        return colreg_type
    
    # check if both cogs are within 15 degrees of each other at end of interaction
    cog_diff_end = front_vessel_cog[-1] - back_vessel_cog[-1]
    if cog_diff_end > 30:
        print('not overtaking')
        return
    # check if back vessel is in front triangle at end of interaction
    shipA_coor = [front_vessel_lon[-1], front_vessel_lat[-1]]
    shipB_coor = [back_vessel_lon[-1], back_vessel_lat[-1]]
    result = is_shipb_front_or_back(shipA_coor, shipB_coor) 
    # True: shipB is in front of shipA
    # False: shipB is behind shipA
    # Null: Not over taking
    if result not True:
        colreg_type = 'NA'
        return colreg_type
    colreg_type = 'NA'
    return colreg_type
    
    

In [201]:
sog_vessel_1_ARRAY = df_v1['SOGS']
cog_vessel_1_ARRAY = df_v1['COGS']
sog_vessel_2_ARRAY = df_v2['SOGS']
cog_vessel_2_ARRAY = df_v2['COGS']
lat_vessel_1_ARRAY = [x[1] for x in df_v1['points']]
lat_vessel_2_ARRAY = [x[1] for x in df_v2['points']]
lon_vessel_1_ARRAY = [x[0] for x in df_v1['points']]
lon_vessel_2_ARRAY = [x[0] for x in df_v2['points']]

# Check if all inputs are defined
# print(len(sog_vessel_1_ARRAY))
# print(len(cog_vessel_1_ARRAY))
# print(len(sog_vessel_2_ARRAY))
# print(len(cog_vessel_2_ARRAY))
# print(len(lat_vessel_1_ARRAY))
# print(len(lat_vessel_2_ARRAY))
# print(len(lon_vessel_1_ARRAY))
# print(len(lon_vessel_2_ARRAY))

label_overtaking(lat_vessel_1_ARRAY, lon_vessel_1_ARRAY,lat_vessel_2_ARRAY,lon_vessel_2_ARRAY,cog_vessel_1_ARRAY,cog_vessel_2_ARRAY,sog_vessel_1_ARRAY, sog_vessel_2_ARRAY)




Vessel 1 is the front vessel
[Decimal('10.5'), Decimal('10.0'), Decimal('10.1'), Decimal('9.7'), Decimal('9.7'), Decimal('9.8'), Decimal('10.1'), Decimal('10.4'), Decimal('9.9'), Decimal('10.2'), Decimal('10.1'), Decimal('10.4'), Decimal('10.4'), Decimal('10.3'), Decimal('10.3'), Decimal('10.7'), Decimal('10.7'), Decimal('10.3'), Decimal('10.0'), Decimal('10.1'), Decimal('9.6'), Decimal('10.1'), Decimal('10.3'), Decimal('10.2'), Decimal('10.0'), Decimal('10.3'), Decimal('10.2'), Decimal('10.4'), Decimal('10.4'), Decimal('10.5'), Decimal('10.3'), Decimal('10.6'), Decimal('10.9'), Decimal('10.9'), Decimal('9.8'), Decimal('9.2'), Decimal('9.2'), Decimal('9.3'), Decimal('8.9'), Decimal('9.0'), Decimal('9.1'), Decimal('9.3'), Decimal('8.3'), Decimal('10.3'), Decimal('11.3'), Decimal('11.8'), Decimal('11.9'), Decimal('11.5'), Decimal('11.5'), Decimal('11.3'), Decimal('11.2'), Decimal('11.3'), Decimal('11.4'), Decimal('11.5')]


TypeError: '>=' not supported between instances of 'int' and 'list'

## Label Crossing colregs

In [181]:
# X1 = lat_vessel_1_ARRAY[0]
# Y1 = lon_vessel_1_ARRAY[0]
# X2 = lat_vessel_1_ARRAY[-1]
# Y2 = lon_vessel_1_ARRAY[-1]
# X3 = lat_vessel_2_ARRAY[0]                      
# Y3 = lon_vessel_2_ARRAY[0]
# X4 = lat_vessel_2_ARRAY[-1]
# Y4 = lon_vessel_2_ARRAY[-1]

lat_vessel_1_ARRAY = [x[1] for x in df_v1['points']]
lat_vessel_2_ARRAY = [x[1] for x in df_v2['points']]
lon_vessel_1_ARRAY = [x[0] for x in df_v1['points']]
lon_vessel_2_ARRAY = [x[0] for x in df_v2['points']]

# lat_vessel_1_ARRAY = [2,4]
# lon_vessel_1_ARRAY = [4,1]
# lat_vessel_2_ARRAY = [1,5]
# lon_vessel_2_ARRAY = [1,3]

# x1 = 2
# y1 = 4
# x2 = 4
# y2 = 1
# x3 = 1                     
# y3 = 1
# x4 = 5
# y4 = 3
def crossing(lat_vessel_1_ARRAY, lon_vessel_1_ARRAY,lat_vessel_2_ARRAY,lon_vessel_2_ARRAY):
    X1 = lat_vessel_1_ARRAY[0]
    Y1 = lon_vessel_1_ARRAY[0]
    X2 = lat_vessel_1_ARRAY[-1]
    Y2 = lon_vessel_1_ARRAY[-1]
    X3 = lat_vessel_2_ARRAY[0]                      
    Y3 = lon_vessel_2_ARRAY[0]
    X4 = lat_vessel_2_ARRAY[-1]
    Y4 = lon_vessel_2_ARRAY[-1]
    
    I1 = [min(X1,X2), max(X1,X2)]
    I2 = [min(X3,X4), max(X3,X4)]
    Ia = [max( min(X1,X2), min(X3,X4) ),min( max(X1,X2), max(X3,X4) )]
    if (max(X1,X2) < min(X3,X4)):
        return False; ## There is no mutual abcisses
    A1 = (Y1-Y2)/(X1-X2) ## Pay attention to not dividing by zero
    A2 = (Y3-Y4)/(X3-X4) ## Pay attention to not dividing by zero
    b1 = Y1-A1*X1 
    b2 = Y3-A2*X3     
    if (A1 == A2):
        colreg_type = 'NA'
        return colreg_type; ## Parallel segments
    Xa = (b2 - b1) / (A1 - A2) ## Once again, pay attention to not dividing by zer
    Ya = A2 * Xa + b2
    if  Xa < max( min(X1,X2), min(X3,X4) ) or Xa > min( max(X1,X2), max(X3,X4) ):
        colreg_type = 'NA'
        return colreg_type; ## intersection is out of bound
    else:
        colreg_type = 'crossing'
        return colreg_type    
    
crossing(lat_vessel_1_ARRAY, lon_vessel_1_ARRAY,lat_vessel_2_ARRAY,lon_vessel_2_ARRAY)

'NA'

## Logic for Julia Flowchart